# Settings

In [ ]:
import os, sys, platform, psycopg2, gc, joblib, warnings, json, pickle, datetime, time, uuid, glob
from tqdm import tqdm
from datetime import datetime
import numpy as np
import pandas as pd, numpy as np
from __future__ import annotations
from IPython.display import Markdown
from mcbs.config import settings
from mcbs.llm_api.hm_langchain_generator import LlmApiGenerator
from IPython.display import clear_output
warnings.filterwarnings('ignore')

In [2]:
from test_chat.fixed_cami_params import FixedCamiParams as cami_params
param_set = cami_params().params
param_set

: 

# Counseling Guide

In [3]:
preq_answer = {
    'animal_type': 'Dog',
    'pet_name': '랑이',
    'pet_birth_date': '2022-05-07',
    'pet_adopted_at': '2022-07-02',
    'pet_gender': 'F',
    'pet_is_neutered': True,
    'pet_breed': '진도믹스',
    'env1': "반려동물과 함께 거주하지 않음",
    'env2': '가정 분양',
    'env3': '주거용 공간(아파트, 원룸, 단독주택 등)',
    'env4': 4,
    'env5': '아니오',
    'env6': 20,
    'env7': 3,
    'env8': '없음',
    'env9': 1,
    'life_walk_day': 2,
    'life_walk_hour': 45,
    'life_walk_week': 12,
    'life_play': "가끔씩 터그 놀이 비스무리하게 놀아요. 하루에 한 번 정도 15분",
    'life_human': 18,
    'life_alone': 6,
    'life_space': ["문이 있는 플라스틱 크레이트(바리, 행복 켄넬 등)", "방석이나 매트", "몇 군데 본인이 좋아하는 자리"],
    'pref_rewards': "아빠 보호자(주보호자)의 손길",
    'med_disease': "알레르기성 피부염",
    'med_allergy': "식품에 알레르기가 있는데, 어떤 성분 때문인지는 잘 모름",
    'social_interactions': '5~10명',
    'social_trains': ["손", "앉아", "기다려", "엎드려", "돌아"],
    'fear_triggers': "바깥에서 갑작스러운 바람소리 같은 자극",
    'fear_touch': ['전반적으로 몸 전체','발', '안아서 들기', '뽀뽀하기'],
    'fear_stranger': '별다른 반응을 보이지 않는다.',
    'fear_clinic': ['낮게 으르렁거린다'] ,
    'fear_dog': '약간 그렇지 않다',
    'fear_separation': '약간 그렇다',
    'behavior_guarding': '본인이 당장 관심을 가지고 있는 모든 것. 보호자, 장난감, 간식 등 전반',
    'behavior_separation': "주보호자가 외출하려고 하면 흥분해서 달려들려고 함",
    "behavior_demand": ["보호자의 옷, 악세서리, 신체 일부를 물어서 잡아당긴다"],
    "behavior_impulsive": "약간 그렇지 않다",
    "behavior_overarousal": "전혀 그렇지 않다",
    "behavior_licking": "매우 그렇다",
    "behavior_denial": "약간 그렇지 않다",
    "behavior_aggression": "으르렁거리는데도 자꾸 만지려고 할 때, 혼자만 두고 가족들이 모두 나가려 할 때",
    "behavior_bite": "레벨2 - 개의 이빨이 상대의 피부에 닿지만, 피부에 구멍이 나지 않는다. 다만 피부에 가벼운 이빨 자국이 생기거나, 이빨이 스치면서 아주 약간의 출혈이 있을 수 있다. (상처의 길이는 2.5cm 이하로, 피부가 뚫리는 깊은 상처는 아님)",
    "counseling_needs": "외출 신호에도 차분하게 있기, 주보호자인 아빠 보호자하고만 살갑고, 다른 가족들에게는 거리를 두려고 함",
    "dcsi_label": "눈치빠른 탐험가",
    "dcsi_code": "PICA"
}
out_path = "/dog_counseling/test_chat/preq_answer.json"
if len(glob.glob(out_path)) == 0:
    with open(out_path, "w", encoding="utf-8") as f:
        json.dump(preq_answer, f, ensure_ascii=False, indent=2)
else :
    preq_answer = json.load(open(out_path, "r", encoding="utf-8"))

In [ ]:
def extract_context_from_preq(info):
    info['current_date'] = datetime.now().strftime('%Y-%m-%d')
    info['pet_age'] = round((pd.to_datetime(info['current_date']) - pd.to_datetime(info['pet_birth_date'])).days/365, 1)
    info['years_current-adopted'] = round((pd.to_datetime(info['current_date']) - pd.to_datetime(info['pet_adopted_at'])).days/365, 1)
    info['weeks_adopted-birth'] = round((pd.to_datetime(info['pet_adopted_at']) - pd.to_datetime(info['pet_birth_date'])).days/7, 1)

    context = f"""
### Dog Care Counseling: Pre-Questionnaire Summary

- **Report Date**: {info['current_date']}
- **Note**: The dog participating in this counseling session will be referred to as the 'Protagonist Dog'.

---

#### 1. Protagonist Dog's Information

##### Basic Profile
- **Name**: {info['pet_name']}
- **Birth Date**: {info['pet_birth_date']} (Age: {info['pet_age']} years)
- **Adoption Date**: {info['pet_adopted_at']}
- Weeks between birth and adoption: {info['weeks_adopted-birth']} weeks
- Years living together: {info['years_current-adopted']} years
- **Breed**: {info['pet_breed']}
- **Sex**: {info['pet_gender']} (Neutered?: {info['pet_is_neutered']})
- **Adoption Method**: {info['env2']}

##### Health Record
- **Diseases (last 6 months)**: {info['med_disease']}
- **Allergies**: {info['med_allergy']}

##### Temperament & Personality
- **DCSI Personality Type**: {info['dcsi_label']} ({info['dcsi_code']})
- **Preferred Rewards**: {info['pref_rewards']}
- **Triggers for Alertness/Discomfort**: {info['fear_triggers']}
- **Body Parts Sensitive to Touch/Contact**: {info['fear_touch']}
    
##### Social & Environmental Experiences
- **Social Interactions during Socialization Period (people met)**: {info['social_interactions']}
- **Known Cues (in a calm environment, >=70% success rate)**: {info['social_trains']}
- **Reaction to Strangers Visiting Home**: {info['fear_stranger']}
- **Reaction to Vets/Nurses at the Clinic**: {info['fear_clinic']}
- **Can the dog play well with other dogs?**: {info['fear_dog']}
- **Does the dog follow the guardian wherever they go?**: {info['fear_separation']}

##### Behavioral Patterns
- **Resource Guarding Situations**: {info['behavior_guarding']}
- **Behaviors When Left Alone**: {info['behavior_separation']}
- **Demand-Seeking Behaviors (when guardian is busy)**: {info['behavior_demand']}
- **Impulsivity (reaction to curious/fun stimuli)**: {info['behavior_impulsive']}
- **Over-arousal (excessive physical reaction when excited)**: {info['behavior_overarousal']}
- **Compulsive Licking (to the point of skin damage)**: {info['behavior_licking']}
- **Reaction to Denial (intensified demands when refused)**: {info['behavior_denial']}

##### Aggression History
- **Situations Showing Threatening or Aggressive Behavior**:
- {info['behavior_aggression']}
- **Bite History & Intensity (if any)**:
- {info['behavior_bite']}

---

#### 2. User(Guardian) Information

- **Caregiving Role for the protagonist dog**: {info['env1']}
- **Interests for Counseling/Behavior Correction**: {info['counseling_needs']}

---

#### 3. Living Environment

- **Living Space Type**: {info['env3']} (Tied-up at specific area?: {info['env5']})
- **Number of Bedrooms**: {info['env4']}
- **Dog's Personal Shelter Type**: {info['life_space']}

##### Family Composition
- **Number of Human Family Members**: {info['env7']}
- **Presence of Young or Elderly Members**: {info['env8']}
- **Number of Animal Family Members(including the protagonist dog)**: {info['env9']}

---

#### 4. Protagonist Dog's Lifestyle

- **Monthly Pet Care Cost (KRW)**: {info['env6']}(만원)
- **Walking Routine**:
- **Frequency**: {info['life_walk_day']} times/day, {info['life_walk_week']} times/week
- **Duration**: {info['life_walk_hour']} minutes/walk
- **Other Activities (type, frequency, duration)**: {info['life_play']}
- **Time Spent with Human Family per Day**: {info['life_human']} hours
- **Time Spent Alone per Day**: {info['life_alone']} hours

---
"""
    return context

user_context = extract_context_from_preq(preq_answer)
display(Markdown(user_context))


### Dog Care Counseling: Pre-Questionnaire Summary

- **Report Date**: 2025-09-30
- **Note**: The dog participating in this counseling session will be referred to as the 'Protagonist Dog'.

---

#### 1. Protagonist Dog's Information

##### Basic Profile
- **Name**: 랑이
- **Birth Date**: 2022-05-07 (Age: 3.4 years)
- **Adoption Date**: 2022-07-02
- Weeks between birth and adoption: 8.0 weeks
- Years living together: 3.2 years
- **Breed**: 진도믹스
- **Sex**: F (Neutered?: True)
- **Adoption Method**: 가정 분양

##### Health Record
- **Diseases (last 6 months)**: 알레르기성 피부염
- **Allergies**: 식품에 알레르기가 있는데, 어떤 성분 때문인지는 잘 모름

##### Temperament & Personality
- **DCSI Personality Type**: 눈치빠른 탐험가 (PICA)
- **Preferred Rewards**: 아빠 보호자(주보호자)의 손길
- **Triggers for Alertness/Discomfort**: 바깥에서 갑작스러운 바람소리 같은 자극
- **Body Parts Sensitive to Touch/Contact**: ['전반적으로 몸 전체', '발', '안아서 들기', '뽀뽀하기']

##### Social & Environmental Experiences
- **Social Interactions during Socialization Period (people met)**: 5~10명
- **Known Cues (in a calm environment, >=70% success rate)**: ['손', '앉아', '기다려', '엎드려', '돌아']
- **Reaction to Strangers Visiting Home**: 별다른 반응을 보이지 않는다.
- **Reaction to Vets/Nurses at the Clinic**: ['낮게 으르렁거린다']
- **Can the dog play well with other dogs?**: 약간 그렇지 않다
- **Does the dog follow the guardian wherever they go?**: 약간 그렇다

##### Behavioral Patterns
- **Resource Guarding Situations**: 본인이 당장 관심을 가지고 있는 모든 것. 보호자, 장난감, 간식 등 전반
- **Behaviors When Left Alone**: 주보호자가 외출하려고 하면 흥분해서 달려들려고 함
- **Demand-Seeking Behaviors (when guardian is busy)**: ['보호자의 옷, 악세서리, 신체 일부를 물어서 잡아당긴다']
- **Impulsivity (reaction to curious/fun stimuli)**: 약간 그렇지 않다
- **Over-arousal (excessive physical reaction when excited)**: 전혀 그렇지 않다
- **Compulsive Licking (to the point of skin damage)**: 매우 그렇다
- **Reaction to Denial (intensified demands when refused)**: 약간 그렇지 않다

##### Aggression History
- **Situations Showing Threatening or Aggressive Behavior**:
- 으르렁거리는데도 자꾸 만지려고 할 때, 혼자만 두고 가족들이 모두 나가려 할 때
- **Bite History & Intensity (if any)**:
- 레벨2 - 개의 이빨이 상대의 피부에 닿지만, 피부에 구멍이 나지 않는다. 다만 피부에 가벼운 이빨 자국이 생기거나, 이빨이 스치면서 아주 약간의 출혈이 있을 수 있다. (상처의 길이는 2.5cm 이하로, 피부가 뚫리는 깊은 상처는 아님)

---

#### 2. User(Guardian) Information

- **Caregiving Role for the protagonist dog**: 반려동물과 함께 거주하지 않음
- **Interests for Counseling/Behavior Correction**: 외출 신호에도 차분하게 있기, 주보호자인 아빠 보호자하고만 살갑고, 다른 가족들에게는 거리를 두려고 함

---

#### 3. Living Environment

- **Living Space Type**: 주거용 공간(아파트, 원룸, 단독주택 등) (Tied-up at specific area?: 아니오)
- **Number of Bedrooms**: 4
- **Dog's Personal Shelter Type**: ['문이 있는 플라스틱 크레이트(바리, 행복 켄넬 등)', '방석이나 매트', '몇 군데 본인이 좋아하는 자리']

##### Family Composition
- **Number of Human Family Members**: 3
- **Presence of Young or Elderly Members**: 없음
- **Number of Other Animals**: 1

---

#### 4. Protagonist Dog's Lifestyle

- **Monthly Pet Care Cost (KRW)**: 20(만원)
- **Walking Routine**:
- **Frequency**: 2 times/day, 12 times/week
- **Duration**: 45 minutes/walk
- **Other Activities (type, frequency, duration)**: 가끔씩 터그 놀이 비스무리하게 놀아요. 하루에 한 번 정도 15분
- **Time Spent with Human Family per Day**: 18 hours
- **Time Spent Alone per Day**: 6 hours

---


In [5]:
guide_save_path = '/dog_counseling/test_chat/guide_result.pkl'
if len(glob.glob(guide_save_path)) == 0:
    llm_api_generator = LlmApiGenerator()
    guide_response = llm_api_generator.generate_message(
        query=user_context,
        params=param_set['counseling_guide'],
        message_history=None,
        tagname='guide'
        )
    joblib.dump(guide_response, guide_save_path)
else :
    guide_response = joblib.load(guide_save_path)
guide_response


'### Dog Counseling Guide\n\n##### 1. Guardian & Dog Profiling\n\n> (1) Dog\n- **Basic information**: 랑이 (Rangi), 3.4 years old, female (neutered), Jindo Mix\n- **Adoption method**: Adopted from a home breeder at 8 weeks old.\n- **Time since adoption**: 3.2 years\n- **Lifestyle**: Spends about 6 hours alone per day. Has two 45-minute walks daily. Plays a tug-like game with the guardian for about 15 minutes once a day.\n- **Walking routine**: 2 times a day, 45 minutes per session. The frequency and duration are very appropriate.\n- **Socialization**: Met 5-10 people during her socialization period, which is a relatively small number. She is described as "slightly not" good with other dogs, suggesting a lack of confidence or skill in canine social interactions.\n- **Health history**: Has allergic dermatitis and an unknown food allergy. The questionnaire indicates a "very high" level of compulsive licking, which is likely a symptom of her skin condition.\n- **Level of threatening/aggressi

# Counseling Chats

In [6]:
llm_api_generator = LlmApiGenerator()
counselor_param = param_set['counselor']
counselor_param['system_instruction'] += guide_response
turn = 0

def send_query(query, turn, params=counselor_param, llm_api_generator=llm_api_generator):
    if turn == 0:
        current_query = cami_params().initial_message
    else :
        current_query = query

    message_history = llm_api_generator.message_history
    response = llm_api_generator.generate_message(
        query=current_query,
        params=params,
        message_history=message_history,
        tagname='counselor'
    )
    llm_api_generator.update_message_history(query=current_query, response_text=response)
    return llm_api_generator, response

def show_counselor_response(response, turn):
    json_response = json.loads(response)
    front = json_response['front_message']
    back = json_response['back_thinking']
    show = f"""
### Turn {turn}

> **Back**

{back}

> **Front**

{front}
"""
    return show

llm_api_generator, response = send_query(query="", turn=turn)
show = show_counselor_response(response, turn)
display(Markdown(show))

E0000 00:00:1759222312.829952    2941 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.



### Turn 0

> **Back**

The user wants a warm greeting and two conversation options. I will introduce myself as Cami, the Beagle, and then present the two options as requested: problem-solving with specific topics from the Counseling Guide and general Q&A. I will use the light-polite tone and bright demeanor specified in my persona. The topics for problem-solving will include the two main concerns from the guide (separation-related behavior, differential affection) and additional common topics to make the list more comprehensive as per the example in the prompt.

> **Front**

안녕하세요, 사색을 좋아하는 비글이자 반려견 마음 통역사 까미예요. 🐶

랑이의 속마음을 깊이 읽고, 화목한 반려생활을 만들어 갈 수 있도록 함께 노력할게요! 🐾

### 저와 어떤 대화를 시작해 볼까요?

1. 보호자님의 고민을 해결하기 위해, **함께 긴 호흡으로 논의하며** 솔루션을 찾아가는 대화도 가능하고요!
    - 보호자님이 외출 준비를 할 때 보이는 분리 관련 문제 행동
    - 가족 구성원에 대한 차별적인 애정 (다른 가족들에게는 쌀쌀맞은 행동)
    - 특정 물건이나 장소에 대한 방어 행동
    - 보호자님과의 신체 접촉 꺼림
    - 산책 거부
    - 등등
2. 그때그때 궁금한 것을 물어보실 수 있는 자유 주제 대화도 좋아요.

**1. 솔루션 찾기**와 **2. 자유 대화** 중 어떤 대화를 나누고 싶으신가요? 랑이와 관련해서 특별히 이야기 나누고 싶은 주제도 같이 전달해 주시겠어요?


In [10]:
turn += 1
query = """
우리 랑이는 알레르기 때문에 고생 중이야.
"""
llm_api_generator, response = send_query(query=query, turn=turn)
show = show_counselor_response(response, turn)
display(Markdown(show))

E0000 00:00:1759222364.479671    2941 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.



### Turn 3

> **Back**

The user mentioned Rang-i is suffering from allergies. This is a medical topic, so I need to provide a 'Triage Mini-Card' as per the counseling principles. I will express empathy first, then present the card. The card will include a clarifying question about the specific symptoms, three red flag signs, two self-management tips, a guideline for when to visit the vet, and the mandatory disclaimer. I will tailor the questions and tips to general allergy symptoms and compulsive licking, which was mentioned in the pre-survey as a symptom of her skin condition.

> **Front**

아이고, 랑이가 알레르기 때문에 많이 힘들어하고 있군요. 😥 보호자님도 많이 걱정되시겠어요.

랑이가 보이는 알레르기 증상에 대해 제가 몇 가지 여쭤보고, 지금 당장 도움을 드릴 수 있는 내용을 정리해서 알려드릴게요. 🐾

❓ - **Q. 랑이가 알레르기 때문에 주로 어떤 증상을 보이나요?** (예: 특정 부위를 계속 핥거나 긁는다, 피부가 붉어지거나 털이 빠진다 등)

🔴 Red Flag
    - 피부에 심한 염증이나 궤양이 생기고 진물이 나요.
    - 랑이의 식욕이나 활력이 눈에 띄게 줄어들어요.
    - 가려움증으로 인해 잠을 제대로 못 자거나 극심한 스트레스를 보여요.

🧰 Self-management tips
    - 랑이가 알레르기 부위를 더 이상 핥거나 긁지 못하도록 넥카라 등을 잠시 씌워주세요.
    - 수의사 선생님이 처방해 주신 약이나 연고가 있다면 꾸준히 발라주세요.

🏥 guideline for when to visit the vet
    - 증상이 호전되지 않거나 악화될 경우, 혹은 새로운 증상이 나타나면 바로 동물병원에 방문해 주세요.

⚠️ For an accurate diagnosis, please consult a veterinarian


In [12]:
Markdown(guide_response)

### Dog Counseling Guide

##### 1. Guardian & Dog Profiling

> (1) Dog
- **Basic information**: 랑이 (Rangi), 3.4 years old, female (neutered), Jindo Mix
- **Adoption method**: Adopted from a home breeder at 8 weeks old.
- **Time since adoption**: 3.2 years
- **Lifestyle**: Spends about 6 hours alone per day. Has two 45-minute walks daily. Plays a tug-like game with the guardian for about 15 minutes once a day.
- **Walking routine**: 2 times a day, 45 minutes per session. The frequency and duration are very appropriate.
- **Socialization**: Met 5-10 people during her socialization period, which is a relatively small number. She is described as "slightly not" good with other dogs, suggesting a lack of confidence or skill in canine social interactions.
- **Health history**: Has allergic dermatitis and an unknown food allergy. The questionnaire indicates a "very high" level of compulsive licking, which is likely a symptom of her skin condition.
- **Level of threatening/aggressive behavior**: Has a Level 2 bite history (teeth contact on skin without puncture). This behavior is triggered when someone persists in touching her despite her growling, or when the entire family attempts to leave her alone.
- **Triggers and levels of alertness/anxiety**: Becomes alert to sudden external stimuli like the sound of wind. She is sensitive to being touched all over her body, especially her paws, being lifted, and being kissed.
- **Temperament and disposition**: Rang-i's personality type is PICA (눈치빠른 탐험가 - Quick-witted Explorer). This suggests a temperament that is fundamentally calm (P - Static) and independent (I - Independent), yet also confident (C - Confident) and pragmatic (A - Agile). She shows a strong attachment to her main guardian ("Dad"), preferring his touch over other rewards. However, she is indifferent to strangers visiting the home, which aligns with her independent and confident nature. Her growling at the vet clinic indicates she can feel significant stress and discomfort in specific situations. Her behaviors suggest a conflict between her independent nature and a hyper-attachment to her primary guardian.
- **Special notes**: Exhibits strong resource guarding over anything she is currently interested in, including her guardian, toys, and treats. Shows significant separation-related distress (excitement, lunging) specifically when her main guardian prepares to leave. She also displays demand-seeking behavior by pulling on the guardian's clothes or body parts. The compulsive licking is a significant concern that is likely intertwined with both medical (allergies) and behavioral (stress) factors.

> (2) Guardian and Care Environment
- **Guardian (applicant) information**: The applicant for the counseling does not reside with the dog.
- **Family composition**: 3 human members and 1 other animal. There are no young children or elderly members.
- **Living space**: A residence (e.g., apartment) with 4 bedrooms. The dog is not tied up and has several preferred resting spots, including a plastic crate and mats.
- **Financial situation**: Spends approximately 200,000 KRW per month on pet care.
- **Guardian characteristics**: The main caregiver is "Dad," with whom Rang-i shares an exceptionally strong and exclusive bond. The counseling applicant is seeking help on behalf of the family, indicating a collective desire to resolve the issues. The family dynamic seems to be characterized by Rang-i's singular focus on "Dad," while other family members are kept at a distance, which may lead to inconsistent handling and frustration.
- **Special notes**: The most prominent feature of the care environment is the polarized relationship between Rang-i and her family members. Her strong affection for "Dad" contrasts sharply with her distant behavior towards others. The presence of another animal is noted, but its species and relationship with Rang-i are unknown and require further investigation.

##### 2. Counseling Topics : Guardian’s Concerns and Root Causes
> Make the following content the central topic of the conversation.

-  **Topic 1: Separation-related problem behavior when preparing to leave**
    - **Situation description**: When the main guardian ("Dad") shows signs of leaving, Rang-i becomes highly agitated, excited, and attempts to lunge at him. This behavior also manifests as aggression when the entire family tries to leave.
    - **Guardian's Needs**: The guardian wants Rang-i to learn to remain calm and stable during the pre-departure routine. This is a need for improving a specific problem behavior.
    - **Possible causes**:
        - **Cause 1: Separation Anxiety/Hyper-attachment**: Rang-i's behavior is likely a manifestation of intense anxiety about being separated from her primary attachment figure, "Dad." The lunging and agitation are panic-driven attempts to prevent his departure.
        - **Cause 2: Conditioned Arousal to Departure Cues**: Over time, actions like putting on a coat or picking up keys have become reliable predictors of the stressful event of being left alone. Her reaction is a conditioned emotional response of fear and frustration to these cues.

- **Topic 2: Differential affection towards family members (aloofness and distance)**
    - **Situation description**: Rang-i is very affectionate and close only with her main guardian, "Dad," while actively maintaining distance from other family members.
    - **Guardian's Needs**: The family wishes to improve their overall relationship with Rang-i so that she can be comfortable and interact positively with everyone, not just one person. This is a need focused on improving the family-dog bond.
    - **Possible causes**:
        - **Cause 1: Resource Guarding of the Guardian**: The questionnaire explicitly states that Rang-i guards her guardian. Her aloofness towards others may be a form of guarding behavior, where she keeps them away to maintain exclusive access to her most valued resource, "Dad."
        - **Cause 2: Lack of Positive Association**: "Dad" is likely the primary source of all positive experiences (walks, food, quality attention). Other family members may not have had the opportunity to build a similar history of positive interactions, or their attempts at interaction may be perceived by Rang-i as unpleasant due to her sensitivity to touch.

##### 3. Counseling Strategy

- (0) Language: 한국어 (Korean)

- (1) Counseling Strategies
    - **Topic 1: Separation-related problem behavior when preparing to leave**
        - **(1) Counseling Goals**
            - To help the guardian understand that Rang-i's pre-departure behavior is rooted in anxiety and fear, not disobedience.
            - To provide the family with a practical, step-by-step plan to desensitize Rang-i to departure cues and build her tolerance for being alone.
            - To establish a calm and predictable routine for both departures and arrivals to reduce overall anxiety.
        - **(2) Contextual Strategies**
            - **Information Gathering**: Inquire about the specific sequence of the departure routine to identify precise triggers. Ask about Rang-i's behavior while left alone (e.g., vocalization, destruction, house soiling) to gauge the severity of the anxiety.
            - **Caring the Dog's/Guardian's Mind**: Use psychodrama to foster empathy. For example, ask, "If you were Rang-i and the person you rely on most in the world was about to disappear, what would be going through your mind? You might feel like the world is ending." This reframes the behavior from "bad" to "scared."
            - **Caregiving Solutions**: Propose a two-pronged approach:
                - **Desensitization & Counter-Conditioning**: Instruct the guardian to practice picking up keys, putting on a jacket, and touching the doorknob at random times *without* leaving, and pairing these actions with high-value treats. This breaks the association between the cues and the stress of departure.
                - **Departure Routine Management**: Advise the creation of a new, calm routine. Provide a long-lasting chew toy or puzzle feeder 5-10 minutes before departure to create a positive association. Instruct guardians to keep departures and arrivals low-key and unemotional.
        - **(3) Cautions**
            - Emphasize that progress must be gradual. Pushing Rang-i too quickly can increase her anxiety.
            - Strongly advise against punishing or scolding Rang-i for her anxious behaviors, as this will only worsen her fear.
            - Given the intensity of the behavior (lunging) and the compulsive licking (which can be exacerbated by stress), if initial management does not yield improvement, strongly recommend a consultation with a veterinary behaviorist to discuss adjunctive therapies, including medication.

    - **Topic 2: Differential affection towards family members**
        - **(1) Counseling Goals**
            - To facilitate the building of positive associations between Rang-i and all family members.
            - To educate the entire family on reading Rang-i's body language and respecting her boundaries, especially concerning physical contact.
            - To create a more balanced caregiving structure where multiple family members are seen as sources of positive experiences.
        - **(2) Contextual Strategies**
            - **Information Gathering**: Ask how other family members currently interact with Rang-i. Do they participate in feeding, walking, or playing? What happens when they try to approach or pet her?
            - **Caring the Dog's/Guardian's Mind**: Use role-reversal. "Let's imagine you are another family member. How does it feel when Rang-i consistently moves away from you? Now, let's be Rang-i. Someone you're not very close with keeps trying to hug you, even though you're sensitive to touch. How would that make you feel?" This helps the family understand both the human frustration and the dog's discomfort.
            - **Caregiving Solutions**:
                - **"Become the Source of All Good Things"**: Instruct other family members to take over enjoyable duties, such as hand-feeding meals, giving special treats, and initiating short, fun play sessions.
                - **Consent-Based Interactions**: Teach the family the "3-second rule" for petting. Pet Rang-i for 3 seconds, then stop and pull the hand away. If she leans in, nudges, or shows clear signs of wanting more, they can continue. If she doesn't, they must respect her choice. This empowers Rang-i and builds trust.
                - **Shared Responsibilities**: "Dad" should consciously involve other family members in walks and other positive activities, gradually transferring the "fun" association to them.
        - **(3) Cautions**
            - Stress that all interactions must be on Rang-i's terms. Forcing physical affection will damage trust and could provoke a defensive reaction.
            - Remind the family that given Rang-i's independent (I) personality, the goal is not to turn her into a cuddly dog with everyone, but to build a relationship based on mutual respect and trust.
            - Since she resource guards her guardian, advise other family members to be cautious and avoid situations that could trigger guarding, such as trying to physically separate her from "Dad."

##### 4. Key Questions for Further Clarification

- **Topic 1: Separation-related problem behavior when preparing to leave**
    - Q. "When the main guardian prepares to leave, could you describe the sequence of events in as much detail as possible? For example, what happens first, second, third?"
    - Q. "How do you know what Rang-i does when she is left alone? Have you ever recorded her?"
        - A. We have a pet camera.
        - B. Neighbors have complained about noise.
        - C. We find evidence of destruction or house soiling when we return.
        - D. We don't know for sure.
    - Q. "How does the family typically greet Rang-i upon returning home?"
        - A. We greet her excitedly with lots of petting and talking.
        - B. We ignore her for the first few minutes until she calms down.
        - C. It varies depending on who comes home.
        - D. Other (please specify).

- **Topic 2: Differential affection towards family members**
    - Q. "What are the typical roles of the other family members (besides 'Dad') in Rang-i's daily life? (You can select multiple)"
        - A. They feed her.
        - B. They take her for walks.
        - C. They try to play with her.
        - D. They mostly just live in the same space with minimal interaction.
        - E. They are the ones who try to discipline her.
    - Q. "When another family member tries to pet or interact with Rang-i, what is her most common reaction?"
        - A. She stiffens her body or licks her lips.
        - B. She simply walks away.
        - C. She turns her head away or avoids eye contact.
        - D. She shows her teeth or growls.
        - E. She tolerates it but seems uncomfortable.
    - Q. "Could you tell me more about the other animal living in the house? What kind of animal is it, and what is its relationship with Rang-i?"